In [2]:
%matplotlib inline


In [3]:
import json
import pprint as pp

In [4]:
# Contains the polling results for every county
# It also contains aggregates per state and also for the entire country
president = json.load(open("president.json"))

# Contains the mapping from county ids to their names
states = json.load(open("en.json"))

# Contains the mapping from candidate id to their names
candidates = json.load(open("president_metadata.json"))

In [12]:
def get_index(entry, name):
    g = [i for i in xrange(0,len(entry['cand'])) if candidates['cands'][entry['cand'][i]]['fn'] == name]
    if len(g) > 0:
        return g[0]
    else:
        return -1
    
def get_county_name(id):
    return states['en']["county." + id]

In [17]:
# To calculate the counties / states with the lowest trump vote
vote_counts = []
for entry in president['results']:
    hillary_index = get_index(entry, 'Hillary Clinton')
    trump_index = get_index(entry, 'Donald Trump')
    vote_count = [entry['vp'][hillary_index], entry['vp'][trump_index], entry['id'], entry['st']]
    if entry['st'] != entry['id']:
        vote_counts.append(vote_count)
        
lowest_hillary = sorted(vote_counts, key=lambda vote_count: vote_count[0])[:10]
lowest_trump = sorted(vote_counts, key=lambda vote_count: vote_count[1])[:10]
def add_county(lst):
    flst = []
    for entry in lst:
        flst.append("%s, %s: %f, %f" % (get_county_name(entry[2]), entry[3], entry[0], entry[1]))
        # entry.append(get_county_name(entry[2]))
    return flst
lowest_hillary = add_county(lowest_hillary)
lowest_trump = add_county(lowest_trump)
print "Schema: County, State: Hillary-Vote Percentage, Trump-Vote Percentage,"
print "Counties with lowest Hillary Clinton Vote Percentage"
for i in lowest_hillary:
    print i
print "Counties with lowest Donald Trump Vote Percentage"
for i in lowest_trump:
    print i


Schema: County, State: Hillary-Vote Percentage, Trump-Vote Percentage,
Counties with lowest Hillary Clinton Vote Percentage
King County, TX: 3.140000, 93.710000
Roberts County, TX: 3.640000, 95.270000
Banner County, NE: 4.620000, 91.030000
Garfield County, MT: 4.750000, 91.200000
McPherson County, NE: 4.900000, 89.860000
Harding County, SD: 4.940000, 90.250000
Piute County, UT: 5.020000, 87.030000
Grant County, NE: 5.080000, 93.150000
Glasscock County, TX: 5.650000, 91.860000
Wallace County, KS: 5.780000, 91.270000
Counties with lowest Donald Trump Vote Percentage
District of Columbia County, DC: 92.850000, 4.120000
Oglala Lakota County, SD: 86.460000, 8.320000
Prince Georges County, MD: 89.330000, 8.340000
Bronx County, NY: 88.730000, 9.590000
San Francisco County, CA: 85.350000, 9.730000
New York County, NY: 87.170000, 9.970000
Petersburg city, VA: 87.520000, 10.560000
Baltimore city, MD: 85.440000, 10.870000
Jefferson County, MS: 86.470000, 12.670000
Clayton County, GA: 85.060000, 1

In [7]:
# To see if results change if candidates consolidate 
# Gary + Trump vs Hillary + Stein
for entry in president['results']:
    # Find all states
    if entry['id'] == entry['st'] and entry['st'] != 'US':
        hillary_index = get_index(entry, 'Hillary Clinton')
        trump_index = get_index(entry, 'Donald Trump')
        gary_index = get_index(entry, 'Gary Johnson')
        jill_index = get_index(entry, 'Jill Stein')
        h_v = entry['vote'][hillary_index]
        t_v = entry['vote'][trump_index]
        g_v = entry['vote'][gary_index] if gary_index != -1 else 0
        j_v = entry['vote'][jill_index] if jill_index != -1 else 0
        
        if h_v > t_v and h_v + j_v < t_v + g_v:
            print "Flipped to Trump:", entry['st'], "; Num Votes:", sum(entry['ev'])
        if h_v < t_v and h_v + j_v > t_v + g_v:
            print "Flipped to Hillary:", entry['st'], "; Num Votes:", sum(entry['ev'])

Flipped to Trump: CO ; Num Votes: 9
Flipped to Trump: ME ; Num Votes: 4
Flipped to Trump: MN ; Num Votes: 10
Flipped to Trump: NH ; Num Votes: 4
Flipped to Trump: NV ; Num Votes: 6


In [14]:
hillary_count, trump_count = 0,0
for entry in president['results']:
    if entry['id'] != entry['st']:
        if entry['total_votes'] > 500000:
            hillary_index = get_index(entry, 'Hillary Clinton')
            trump_index = get_index(entry, 'Donald Trump')
            h_v = entry['vote'][hillary_index]
            t_v = entry['vote'][trump_index]
            if t_v > h_v:
                trump_count += 1
                print get_county_name(entry['id']), entry['st']
            else:
                hillary_count += 1
print "Trump Won: ", trump_count
print "Hillary Won: ", hillary_count

Maricopa County AZ
Suffolk County NY
Tarrant County TX
Trump Won:  3
Hillary Won:  33
